# Prediction using Text Features

In miletone 3, we applied SVM and random forest to predict genre from overview and title term frequency, but the test accuracy stuck on about 20%. In milestone 4, we also improved the prediction by modeling the text features from overview of movies. In the following part, we:

* Extract features using TF-IDF instead of term frequency from overview, title and tagline
* Prescreening the features using chi-square value


The new prediction accuracy on the same test set reaches ~46%. 

In [48]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
import imdb
import numpy                  as np
import pandas                 as pd
import scipy                  as sp
import sklearn.neighbors      as knn
import matplotlib
import matplotlib.pyplot      as plt
import seaborn
import requests
import urllib
import joblib
import requests
import json
import os
import random
import seaborn as sns
#import statsmodels.api as sm
from ast                                  import literal_eval
from matplotlib                           import rcParams
from sklearn                              import discriminant_analysis
from sklearn.decomposition                import PCA
from sklearn                              import preprocessing
from sklearn.linear_model                 import LogisticRegression as LogReg
from sklearn.discriminant_analysis        import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis        import QuadraticDiscriminantAnalysis as QDA
from scipy.stats                          import mode
from sklearn                              import linear_model
from IPython.core.interactiveshell        import InteractiveShell
from time                                 import sleep
from collections                          import Counter
from itertools                            import combinations, permutations
from urlparse                             import urljoin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split as sk_split

In [50]:
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [51]:
matplotlib.style.use('ggplot')
rcParams['figure.figsize'] = (20, 10)
rcParams['axes.facecolor'] = "w"
rcParams['grid.color'] = "gray"
rcParams['grid.linewidth'] = 0.5

## Clean the data

In [52]:
def extract_first_genre(x):
    try:
        return [genre['name'] for genre in literal_eval(x)][0]
    except Exception:
        return 0


In [53]:
merged_mdb_final = pd.read_csv('merged_mdb_final.txt')
merged_mdb_final = merged_mdb_final.drop('Unnamed: 0', axis=1)

In [54]:
#merged_mdb_final['genres'].unique()
#Pick the first genre randomly to deal with the problem as a multiclass prediction
merged_mdb_final['final_genre'] = merged_mdb_final['genres'].map(lambda x: extract_first_genre(x))

In [55]:
filtered_merged_mdb_final = merged_mdb_final[~merged_mdb_final.final_genre.isin((0,'War', 'Western', 
                                                                                'TV Movie', 'Foreign',
                                                                               'History', 'Music'))].copy()

In [56]:
tmp = filtered_merged_mdb_final['final_genre']
#print len(tmp[0])
#Convert Y variable to numerical value prior to modelling
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

tmp1 = le.fit_transform(tmp)
print le.classes_
print len(tmp1)
yy = pd.DataFrame(tmp1)
#merged_mdb_uni2['uni_genre_x'] = le.transform(merged_mdb_uni2['uni_genre_x']) 

[u'Action' u'Adventure' u'Animation' u'Comedy' u'Crime' u'Documentary'
 u'Drama' u'Family' u'Fantasy' u'Horror' u'Mystery' u'Romance'
 u'Science Fiction' u'Thriller']
4250


In [57]:
data1 = filtered_merged_mdb_final[['final_genre', 'overview', 'title_x', 'tagline']]

In [58]:
print filtered_merged_mdb_final.columns

Index([u'title_x', u'imdb_id', u'id', u'overview', u'budget', u'genres',
       u'release_date', u'revenue', u'runtime', u'original_language',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'spoken_languages', u'tagline',
       u'vote_average', u'vote_count', u'title_y', u'rating', u'votes',
       u'year', u'cast', u'director', u'writer', u'final_genre'],
      dtype='object')


In [59]:
print filtered_merged_mdb_final[['title_x', 'genres', 'tagline']].iloc[4, 1]

[{u'id': 28, u'name': u'Action'}, {u'id': 878, u'name': u'Science Fiction'}]


## Extract TF-IDF features

In [60]:
## Extract TF-IDF uni-gram from overview
vectorizer = TfidfVectorizer(stop_words='english',decode_error='ignore',analyzer='word', min_df=0, max_df=0.9)

corpus = filtered_merged_mdb_final['overview'].values.astype('U')

wordvec = vectorizer.fit_transform(corpus.ravel())
wordvec = wordvec.toarray()
words = vectorizer.get_feature_names()
print("number of words in plot summary: ", len(words))
overview_db = pd.DataFrame(wordvec,columns=words)



('number of words in plot summary: ', 20216)


In [61]:
## Extract TF-IDF uni-gram from title
vectorizer = TfidfVectorizer(stop_words='english',decode_error='ignore',analyzer='word', min_df=0, max_df=0.9)

corpus = filtered_merged_mdb_final['title_x'].values.astype('U')

wordvec = vectorizer.fit_transform(corpus.ravel())
wordvec = wordvec.toarray()
words = vectorizer.get_feature_names()
print("number of words in plot summary: ", len(words))
title_db = pd.DataFrame(wordvec,columns=words)


('number of words in plot summary: ', 4178)


In [62]:
## Extract TF-IDF uni-gram from overview
vectorizer = TfidfVectorizer(stop_words='english',decode_error='ignore',analyzer='word', min_df=0, max_df=0.9)

corpus = filtered_merged_mdb_final['tagline'].values.astype('U')

wordvec = vectorizer.fit_transform(corpus.ravel())
wordvec = wordvec.toarray()
words = vectorizer.get_feature_names()
print("number of words in plot summary: ", len(words))
tagline_db = pd.DataFrame(wordvec,columns=words)



('number of words in plot summary: ', 3782)


## Prescreening using Chi-square test
We find that the prediction performance is best if we only remove the extremely nonclassifiable features, so we filter out those with p-value > 0.999 (this is not a routing statistical test which usually remove the features with p-value < 0.05) 

In [63]:
## Prescreening the words using chi-square for overview
chi2_val, p_val = chi2(X=overview_db.values, y=tmp)
xx_overview = overview_db.iloc[:, p_val<=0.997]
## Prescreening the words using chi-square for title
chi2_val, p_val = chi2(X=title_db.values, y=tmp)
xx_title = title_db.iloc[:, p_val<=0.99]
## Prescreening the words using chi-square for title
chi2_val, p_val = chi2(X=tagline_db.values, y=tmp)
xx_tagline = tagline_db.iloc[:, p_val<=0.9]


In [64]:
## Build X-Y data for modeling
data_overview = pd.merge(yy, xx_overview, left_index=True, right_index=True)
data_title = pd.merge(yy, xx_title, left_index=True, right_index=True)
data_tagline = pd.merge(yy, xx_tagline, left_index=True, right_index=True)
#data2 = pd.merge(data2, zz, left_index=True, right_index=True)
data_text = pd.merge(yy, xx_overview, left_index=True, right_index=True)
data_text = pd.merge(data_text, xx_title, left_index=True, right_index=True)
data_text = pd.merge(data_text, xx_tagline, left_index=True, right_index=True)

## Modeling for Overview Data

In [65]:
from sklearn.cross_validation import train_test_split as sk_split
# Split final version of data into training/test data sets
data_train, data_test = sk_split(data_overview, test_size = 0.3, random_state=123)
trainX = data_train.values[:, 1:]
trainY = data_train.values[:, 0]
testX = data_test.values[:, 1:]
testY = data_test.values[:, 0]


In [73]:
model_logistic = LogReg(random_state=10, C=10, class_weight='balanced')
model_logistic.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', model_logistic.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', model_logistic.score(X=testX, y=testY)
trainY_pred = model_logistic.predict_proba(X=trainX)
testY_pred = model_logistic.predict_proba(X=testX)


LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=10,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Training accuracy:	0.906554621849
Test accuracy:		0.456470588235


### We can see the test accuracy using simple logistic regression can reach 45.6%. The confusion matrix is shown below. 

In [35]:
## Compute confusion matrix
testY_hat = model_logistic.predict(X=testX)
pd.DataFrame(data=confusion_matrix(y_true=testY, y_pred=testY_hat), index=le.classes_, columns=le.classes_)

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Horror,Mystery,Romance,Science Fiction,Thriller
Action,123,3,3,35,3,0,53,0,0,5,0,0,0,0
Adventure,17,13,3,18,0,1,16,1,0,3,0,0,0,1
Animation,13,2,6,17,0,2,8,2,0,2,0,0,0,0
Comedy,26,1,1,153,0,0,58,2,0,7,0,1,0,0
Crime,13,1,0,12,4,0,24,0,0,3,2,0,0,5
Documentary,3,2,0,8,0,6,14,0,0,0,0,0,0,0
Drama,26,3,0,78,1,0,175,0,0,12,1,1,0,1
Family,5,2,2,8,0,0,6,0,0,3,0,0,0,0
Fantasy,2,2,0,14,0,0,10,0,3,1,0,0,1,0
Horror,11,4,0,17,0,0,24,0,0,34,1,0,0,3


In [36]:
model_lsvm = LinearSVC(random_state=10, C=1,class_weight='balanced')
model_lsvm.fit(X=trainX, y=trainY)
model_lsvm.score(X=trainX, y=trainY)
model_lsvm.score(X=testX, y=testY)

LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=10, tol=0.0001,
     verbose=0)

0.91764705882352937

0.42980392156862746

### We also tried linear SVM but the test accuracy is slight lower than logistic regression.

In [37]:
dt_stump = DecisionTreeClassifier(max_depth=20, max_features='auto')
dt_stump.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', dt_stump.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', dt_stump.score(X=testX, y=testY)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Training accuracy:	0.345546218487
Test accuracy:		0.256470588235


In [38]:
model_adaboost = AdaBoostClassifier(base_estimator=dt_stump, n_estimators=100, random_state=10)
model_adaboost.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', model_adaboost.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', model_adaboost.score(X=testX, y=testY)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=100, random_state=10)

Training accuracy:	0.746554621849
Test accuracy:		0.276078431373


### The performance of decision tree and adaboost is disappointing in this problem.

## Ensemble with Deep Learning on Poster Images
We combine the results from overview text and the results from deep learning for poster images. The final test accuracy is 47.4%. 

In [129]:
train_pred_poster = pd.read_csv('train_pred.csv')
test_pred_poster = pd.read_csv('test_pred.csv')
#testYY = np.concatenate((testY_pred, test_pred_poster.values), axis=1)
alpha = 0.76
testYY2 = alpha*testY_pred + (1-alpha) * test_pred_poster.values
testY_pred1 = np.argmax(testYY2, axis=1)
print 'Accuracy of ensemble: ', np.mean(testY_pred1 == testY)

Accuracy of ensemble:  0.472156862745


## Modeling for Title Data

In [39]:
from sklearn.cross_validation import train_test_split as sk_split
# Split final version of data into training/test data sets
data_train, data_test = sk_split(data_title, test_size = 0.3, random_state=123)
trainX = data_train.values[:, 1:]
trainY = data_train.values[:, 0]
testX = data_test.values[:, 1:]
testY = data_test.values[:, 0]


In [40]:
model_logistic = LogReg(random_state=10, C=0.1,class_weight='balanced')
model_logistic.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', model_logistic.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', model_logistic.score(X=testX, y=testY)

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=10,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Training accuracy:	0.426890756303
Test accuracy:		0.276078431373


## Modeling for Tagline Data

In [41]:
## Prescreening the words using chi-square for title
chi2_val, p_val = chi2(X=tagline_db.values, y=tmp)
xx_tagline = tagline_db.iloc[:, p_val<=0.9]
print xx_tagline.shape
data_tagline = pd.merge(yy, xx_tagline, left_index=True, right_index=True)

(4250, 1231)


In [42]:
#from sklearn.cross_validation import train_test_split as sk_split
# Split final version of data into training/test data sets
data_train, data_test = sk_split(data_tagline, test_size = 0.3, random_state=123)
trainX = data_train.values[:, 1:]
trainY = data_train.values[:, 0]
testX = data_test.values[:, 1:]
testY = data_test.values[:, 0]


In [43]:
model_logistic = LogReg(random_state=10, C=10, class_weight='balanced')
model_logistic.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', model_logistic.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', model_logistic.score(X=testX, y=testY)

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=10,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Training accuracy:	0.522352941176
Test accuracy:		0.233725490196


We see the prediction accuracy using tapline data is only about 23%. 

## Modeling using Text Data from Overview, Title and Tagline

In [44]:
#from sklearn.cross_validation import train_test_split as sk_split
# Split final version of data into training/test data sets
data_train, data_test = sk_split(data_text, test_size = 0.3, random_state=123)
trainX = data_train.values[:, 1:]
trainY = data_train.values[:, 0]
testX = data_test.values[:, 1:]
testY = data_test.values[:, 0]


In [45]:
model_logistic = LogReg(random_state=10, C=10, class_weight='balanced')
model_logistic.fit(X=trainX, y=trainY)
print 'Training accuracy:\t', model_logistic.score(X=trainX, y=trainY)
print 'Test accuracy:\t\t', model_logistic.score(X=testX, y=testY)

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=10,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Training accuracy:	0.97512605042
Test accuracy:		0.439215686275


We see the performance using Overview, Title and Tagline features can not reach the performance using overview feature only. 

## In the following part, we tried to use both uni-gram and bi-gram for prediction. 

In [46]:
tfidf = TfidfVectorizer(stop_words='english',decode_error='ignore',analyzer='word', min_df=0, max_df=0.9, ngram_range=(1,2))
corpus1 = data1['overview'].values.astype('U')
wordvec1 = tfidf.fit_transform(corpus1.ravel())
wordvec1 = wordvec1.toarray()
words1 = tfidf.get_feature_names()
print("number of words in plot summary: ", len(words1))
overview_db1 = pd.DataFrame(wordvec1,columns=words1)

chi2_val, p_val = chi2(X=overview_db1.values, y=tmp)
xx = overview_db1.iloc[:, p_val<=1-1e-5]
#xx = overview_db1
data_bigram = pd.merge(yy, xx, left_index=True, right_index=True)
print len(data_bigram.columns)

data_train1, data_test1 = sk_split(data_bigram, test_size = 0.3, random_state=123)
trainX1 = data_train1.values[:, 1:]
trainY1 = data_train1.values[:, 0]
testX1 = data_test1.values[:, 1:]
testY1 = data_test1.values[:, 0]


('number of words in plot summary: ', 124081)
53622


In [47]:
model_logistic1 = LogReg(random_state=10, C=10,class_weight='balanced')
model_logistic1.fit(X=trainX1, y=trainY1)
model_logistic1.score(X=trainX1, y=trainY1)
model_logistic1.score(X=testX1, y=testY1)

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=10,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

0.96773109243697475

0.41254901960784313

We notice that the prediction accuracy using bi-gram is not as good as that using uni-gram.